In [0]:
#This Code predict Label for Corefx Issues with Title

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np
import re

url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

data = pd.read_csv('IssueDetailscorefx_20102019.csv', encoding='latin-1')

y = list(data['Label']) 
x = list(data['Title_Description']) 


for idx in range(len(x)):
   x[idx] = re.sub(r'[^a-zA-Z]'," ",x[idx])
  
le = preprocessing.LabelEncoder()
le.fit(y)

categories = list(set(y))

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le,categories)

untest = decode(le, test)


x_enc = x
y_enc = encode(le, y)

x_train = np.asarray(x_enc[:800])
y_train = np.asarray(y_enc[:800])


x_test = np.asarray(x_enc[:800])
y_test = np.asarray(y_enc[:800])

In [36]:
x_test

array(['Making debugging of packageTesting easier  Instead of always manually passing the LocalPackagesPath this will pass the default path where generally local built packages are kept ',
       'Possible overflow in Partitioner        Fixes https   github com dotnet corefx issues      ',
       'Update branding for Microsoft Netcore Platforms       ',
       'Update branding for Microsoft Netcore Platforms       ',
       ' release      Update dependencies from dotnet coreclr This pull request updates the following dependencies   marker       Begin      e e bd    f e    f   d   bc         From https   github com dotnet coreclr     Build                   Date Produced                   PM     Commit         baddeb e   d      a    e                Branch    refs heads release         Updates          Microsoft NET Sdk IL            rc                Microsoft NETCore ILAsm            rc                Microsoft NETCore Runtime CoreCLR            rc            marker       End      e e

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [39]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(len(categories), activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [40]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=3, batch_size=32)
    model.save_weights('./model_corefx_TitleDescription_Label_Prediction.h5')

Epoch 1/3
800/800 [==============================] - 75s 94ms/step - loss: 5.1454 - acc: 0.2512
Epoch 2/3
800/800 [==============================] - 66s 83ms/step - loss: 4.1075 - acc: 0.2650
Epoch 3/3
800/800 [==============================] - 66s 83ms/step - loss: 3.8096 - acc: 0.3063


In [0]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model_corefx_TitleDescription_Label_Prediction.h5')  
    predicts = model.predict(x_test, batch_size=32)  

In [45]:
y_test = np.asarray(y_enc[:800])
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))

                                                                        precision    recall  f1-score   support

                                                          * NO MERGE *       0.00      0.00      0.00         3
                                                * NO MERGE *|NO REVIEW       0.00      0.00      0.00         1
            * NO MERGE *|NO REVIEW|api-suggestion|area-System.Net.Http       0.00      0.00      0.00         1
                                 * NO MERGE *|NO REVIEW|area-System.IO       0.00      0.00      0.00         1
                                       * NO MERGE *|Servicing-approved       0.00      0.00      0.00         1
                       * NO MERGE *|Servicing-approved|area-System.Net       0.00      0.00      0.00         2
                                  * NO MERGE *|area-Meta|documentation       0.00      0.00      0.00         1
                                      * NO MERGE *|area-System.Drawing       0.00      0.00      0.00  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# New Section